In [12]:
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import itertools
import csv , codecs
import glob
from PIL import Image
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA



In [13]:
data_dir = "./EEG_data/"

## CSV 파일 생성 함수


### task 0 1 2 : 1-back,  2-back,  rest

### eye 0  1    : close eye,     open eye

### respiratory 0 1 2 : mouse, nose, O2 



In [ ]:
def dataset_csv_3():
    normalize_power = str(input("Relative, Abs :"))
    excel_dir = "./EEG_data" + '/' + normalize_power
    excel_list = sorted(glob.glob(excel_dir+'/[!~]*.xlsx'))
    
    subject = int(input("subject :"))
    book = Workbook()
    sheet = book.active
    sheet.cell(row=1, column=161).value = 'target'
 
    # x축 생성   
    for i in range(1,33):
        sheet.cell(row=1, column=i).value = 'ACh' + str(i)
        sheet.cell(row=1, column=i+32).value = 'BCh' + str(i)
        sheet.cell(row=1, column=i+64).value = 'DCh' + str(i)
        sheet.cell(row=1, column=i+96).value = 'GCh' + str(i)
        sheet.cell(row=1, column=i+128).value = 'TCh' + str(i)
        
    # 파일을 순차적으로 열어서 셀 영역을 복사
    for j in range(0,5):
        wb = openpyxl.load_workbook(excel_list[j], data_only=True)
        wb_sheet = wb.sheetnames
        for k in range(0,12):
            source = wb[wb_sheet[k]]
            for l in range(2, 22):
                for m in range(2, 34):
                    sheet.cell(row=20*k+l,column=32*j+m-1).value = source.cell(row=l,column=m).value
        

    with open('./dataset_eeg_3.csv', 'w', newline="") as f:
        c = csv.writer(f)
        for r in sheet.rows:
            c.writerow([cell.value for cell in r])
            
    df = pd.read_csv('./dataset_eeg_3.csv')
    df[df =='                      NaN'] =np.nan
    df = df.fillna(0.0).astype('float64')
    pd.options.mode.chained_assignment = None
#1-back mouse, nose, o2
    df["target"][0*subject:1*subject] = "0" #0:20
    df["target"][1*subject:2*subject] = "1" #21:40
    df["target"][2*subject:3*subject] = "2" #41:60
#2-back mouse, nose, o2
    df["target"][3*subject:4*subject] = "0" #60:80
    df["target"][4*subject:5*subject] = "1" #80:100
    df["target"][5*subject:6*subject] = "2" #100:120
#rest close eye mouse, nose, o2
    df["target"][6*subject:7*subject] = "0" #120:140
    df["target"][7*subject:8*subject] = "1" #140:160
    df["target"][8*subject:9*subject] = "2" #160:180
#rest open eye mouse, nose, o2
    df["target"][9*subject:10*subject] = "0" #180:200
    df["target"][10*subject:11*subject] = "1" #200:220
    df["target"][11*subject:12*subject] = "2" #220:240
    df.astype('float64')
    df.to_csv('./dataset_eeg_3.csv')
    
    return df
            


In [ ]:
eeg_data_3 = dataset_csv_3()

In [16]:
state={}

state["1back_mouse"]=eeg_data_3.iloc[0:20]
state["1back_nose"]=eeg_data_3.iloc[20:40]
state["1back_O2"]=eeg_data_3.iloc[40:60]

state["2back_mouse"]=eeg_data_3.iloc[60:80]
state["2back_nose"]=eeg_data_3.iloc[80:100]
state["2back_O2"]=eeg_data_3.iloc[100:120]

state["close_rest_mouse"]=eeg_data_3.iloc[120:140]
state["close_rest_nose"]=eeg_data_3.iloc[140:160]
state["close_rest_O2"]=eeg_data_3.iloc[160:180]

state["open_rest_mouse"]=eeg_data_3.iloc[180:200]
state["open_rest_nose"]=eeg_data_3.iloc[200:220]
state["open_rest_O2"]=eeg_data_3.iloc[220:240]

In [17]:
ds_train = {}
ds_test = {}


ds_train={"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
       
       
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
       "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
       "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])
       
       
      }

ds_test = {"1back_mouse":state["1back_mouse"], 
       "1back_nose":state["1back_nose"],
       "1back_O2":state["1back_O2"],
       "1back_m_n":state["1back_mouse"].append(state["1back_nose"]),
       "1back":state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]),
       
                                               
       "2back_mouse":state["2back_mouse"],
       "2back_nose":state["2back_nose"],
       "2back_O2":state["2back_O2"],
       "2back_m_n":state["2back_mouse"].append(state["2back_nose"]),
       "2back":state["2back_mouse"].append(state["2back_nose"]).append(state["2back_O2"]),
       
       "close_rest_mouse":state["close_rest_mouse"], 
       "close_rest_nose":state["close_rest_nose"],
       "close_rest_O2":state["close_rest_O2"],
       "close_rest_m_n":state["close_rest_mouse"].append(state["close_rest_nose"]),
       "close_rest":state["close_rest_mouse"].append(state["close_rest_nose"]).append(state["close_rest_O2"]),
       
       "open_rest_mouse":state["open_rest_mouse"],
       "open_rest_nose":state["open_rest_nose"],
       "open_rest_O2":state["open_rest_O2"],
       "open_rest_m_n":state["open_rest_mouse"].append(state["open_rest_nose"]),
       "open_rest":state["open_rest_mouse"].append(state["open_rest_nose"]).append(state["open_rest_O2"]),
        
        
               
       "total_mouse" : state["1back_mouse"].append(state["2back_mouse"]).append(state["close_rest_mouse"]).append(state["open_rest_mouse"]),
        "total_nose" : state["1back_nose"].append(state["2back_nose"]).append(state["close_rest_nose"]).append(state["open_rest_nose"]),
       "total_O2" : state["1back_O2"].append(state["2back_O2"]).append(state["close_rest_O2"]).append(state["open_rest_O2"]),
       "total_m_n": state["1back_mouse"].append(state["1back_nose"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]),
        "total": state["1back_mouse"].append(state["1back_nose"]).append(state["1back_O2"]).append(state["2back_mouse"]).append(state["2back_nose"]).append(state["2back_O2"]).append(state["close_rest_mouse"]).append(state["close_rest_nose"]).append(state["close_rest_O2"]).append(state["open_rest_mouse"]).append(state["open_rest_nose"]).append(state["open_rest_O2"])

       }




In [18]:
def LDA_RF_M_N_O2 (tasks, k_parameter, num_iter):

#     for mode in ['no_lda','lda']:
    for mode in ['lda']:
        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-1])
            Y = np.array(train_3['target'])

            rnd_clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)

            all_scores = []
            Mouse = 0
            Nose = 0
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y):

                    X_final = X
                    lda = LinearDiscriminantAnalysis(n_components=1)
                    if(mode == 'lda') : 
                        lda.fit(X,Y)
#                         X_final = X
                        X_final = lda.transform(X)
                    

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                    test = np.array(test_3.iloc[:,0:-1])
                    if mode == 'lda':
#                         pass
                        test = lda.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa=='0')
                    Nose = Nose+ np.count_nonzero(aaa=='1')

            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))

In [19]:
def PCA_RF_M_N_O2 (tasks, k_parameter, num_iter):

#     for mode in ['no_lda','lda']:
    for mode in ['pca']:
        for task_train,task_test in tasks:

            print('-------------------')
            print('training:{}/test:{}/mode:{}'.format(task_train,task_test,mode))
            train_3 = ds_train[task_train]
            test_3  = ds_test[task_test]

            X = np.array(train_3.iloc[:,0:-1])
            Y = np.array(train_3['target'])

            rnd_clf = RandomForestClassifier(n_estimators=1000,max_depth= None, n_jobs=-1,class_weight="balanced" )
#             rnd_clf = RandomForestClassifier(n_estimators=1000,max_features= 160,n_jobs=-1 )

            all_scores = []
            Mouse = 0
            Nose = 0
            for idx in range(num_iter):
                skf = StratifiedKFold(n_splits=k_parameter, shuffle=True)
                skf.get_n_splits(X,Y)

                for train_index, test_index in skf.split(X,Y):

                    X_final = X
                    pca = PCA(n_components=0.8)
                    if(mode == 'pca') : 
                        pca.fit(X,Y)
                        X_final = pca.transform(X)
#                         X_final = X
                    

                    rnd_clf.fit(X_final[train_index], Y[train_index])
                    val_mae = rnd_clf.score(X_final[test_index],Y[test_index])
                    all_scores.append(val_mae)

                    test = np.array(test_3.iloc[:,0:-1])
                    if mode == 'pca':
#                         pass
                        test = pca.transform(test)

                    aaa = rnd_clf.predict(test)

                    Mouse = Mouse+np.count_nonzero(aaa=='0')
                    Nose = Nose+ np.count_nonzero(aaa=='1')

            print('model_score:{}'.format(np.mean(all_scores)))
            print('mouse:{}({:.2}), nose:{}({:.2}) / total:{}'.format(Mouse,Mouse/(Mouse+Nose),Nose,Nose/(Mouse+Nose),(Mouse+Nose)))

# Task list

        * 1back_mouse, 1back_nose, 1back_O2, 1back_m_n, 1back
        
        
        * 2back_mouse, 2back_nose, 2back_O2, 2back_m_n, 2back
        
        
        * close_rest_mouse, close_rest_nose, close_rest_O2, close_rest_m_n, close_rest
        
        
        * open_rest_mouse, open_rest_nose, open_rest_O2, open_rest_m_n, open_rest
        
        * total_mouse, total_nose, total_O2, total_m_n
 


### Poster Result data

In [20]:
print("Poster Result data")
tasks1 = [
    ('close_rest_m_n','close_rest_O2'),
    ('close_rest_m_n','open_rest_O2'),
     ('close_rest_m_n','1back_O2'),
     ('close_rest_m_n','2back_O2'),

 ]
print("LDA-RF")
LDA_RF_M_N_O2(tasks1,k_parameter = 10,num_iter = 1)


Poster Result data
LDA-RF
-------------------
training:close_rest_m_n/test:close_rest_O2/mode:lda


ValueError: n_components cannot be larger than min(n_features, n_classes - 1).

In [21]:
tasks1 = [
    ('close_rest_m_n','close_rest_mouse'),
    ('close_rest_m_n','open_rest_mouse'),
     ('close_rest_m_n','1back_mouse'),
     ('close_rest_m_n','2back_mouse'),
    
    ('close_rest_m_n','close_rest_nose'),
    ('close_rest_m_n','open_rest_nose'),
     ('close_rest_m_n','1back_nose'),
     ('close_rest_m_n','2back_nose'),

 ]
print("LDA-RF")
LDA_RF_M_N_O2(tasks1,k_parameter = 10,num_iter = 1)


LDA-RF
-------------------
training:close_rest_m_n/test:close_rest_mouse/mode:lda


ValueError: n_components cannot be larger than min(n_features, n_classes - 1).

In [ ]:
  
tasks3 = [
    ('total_m_n','open_rest_mouse'),
    ('total_m_n','close_rest_mouse'),
     ('total_m_n','1back_mouse'),
     ('total_m_n','2back_mouse'),
    
    
    ('total_m_n','open_rest_nose'),
    ('total_m_n','close_rest_nose'),
     ('total_m_n','1back_nose'),
     ('total_m_n','2back_nose'),
 ]
print("LDA-RF")
LDA_RF_M_N_O2(tasks3,k_parameter = 10,num_iter = 1)
print("------------------------------------------")
print("------------------------------------------")
print("------------------------------------------")
print("PCA-RF")
PCA_RF_M_N_O2(tasks3,k_parameter = 10,num_iter = 1)

LDA-RF
-------------------
training:total_m_n/test:open_rest_mouse/mode:lda
model_score:0.98125
mouse:0(0.0), nose:200(1.0) / total:200
-------------------
training:total_m_n/test:close_rest_mouse/mode:lda
model_score:0.99375
mouse:200(1.0), nose:0(0.0) / total:200
-------------------
training:total_m_n/test:1back_mouse/mode:lda
model_score:0.99375
mouse:200(1.0), nose:0(0.0) / total:200
-------------------
training:total_m_n/test:2back_mouse/mode:lda
model_score:0.99375
mouse:200(1.0), nose:0(0.0) / total:200
-------------------
training:total_m_n/test:open_rest_nose/mode:lda
model_score:0.96875
mouse:1(0.005), nose:199(0.99) / total:200
-------------------
training:total_m_n/test:close_rest_nose/mode:lda


In [ ]:
tasks4 = [
    ('open_rest_m_n', 'open_rest_O2'),
    ('open_rest_m_n','1back_O2'),
    ('open_rest_m_n','2back_O2'),
    ('open_rest_m_n','close_rest_O2'),
]

print("LDA-RF")
LDA_RF_M_N_O2(tasks4,k_parameter = 10,num_iter = 1)
print("------------------------------------------")
print("------------------------------------------")
print("------------------------------------------")
print("PCA-RF")
PCA_RF_M_N_O2(tasks4,k_parameter = 10,num_iter = 1)

LDA-RF
-------------------
training:open_rest_m_n/test:open_rest_O2/mode:lda
model_score:0.9
mouse:119(0.59), nose:81(0.41) / total:200
-------------------
training:open_rest_m_n/test:1back_O2/mode:lda
model_score:0.875
mouse:82(0.41), nose:118(0.59) / total:200
-------------------
training:open_rest_m_n/test:2back_O2/mode:lda
model_score:0.875
mouse:70(0.35), nose:130(0.65) / total:200
-------------------
training:open_rest_m_n/test:close_rest_O2/mode:lda
model_score:0.875
mouse:158(0.79), nose:42(0.21) / total:200
------------------------------------------
------------------------------------------
------------------------------------------
PCA-RF
-------------------
training:open_rest_m_n/test:open_rest_O2/mode:pca
model_score:0.3
mouse:91(0.46), nose:109(0.55) / total:200
-------------------
training:open_rest_m_n/test:1back_O2/mode:pca
model_score:0.225
mouse:119(0.59), nose:81(0.41) / total:200
-------------------
training:open_rest_m_n/test:2back_O2/mode:pca
model_score:0.325
mo

In [ ]:
tasks5 = [
    ('open_rest_m_n', 'open_rest_mouse'),
    ('open_rest_m_n','close_rest_mouse'),
    ('open_rest_m_n','1back_mouse'),
    ('open_rest_m_n','2back_mouse'),
    
    
    ('open_rest_m_n','open_rest_nose'),
    ('open_rest_m_n','close_rest_nose'),
    ('open_rest_m_n','1back_nose'),
    ('open_rest_m_n','2back_nose'),
]
print("LDA-RF")
LDA_RF_M_N_O2(tasks5,k_parameter = 10,num_iter = 1)
print("------------------------------------------")
print("------------------------------------------")
print("------------------------------------------")
print("PCA-RF")
PCA_RF_M_N_O2(tasks5,k_parameter = 10,num_iter = 1)


LDA-RF
-------------------
training:open_rest_m_n/test:open_rest_mouse/mode:lda
model_score:0.875
mouse:197(0.98), nose:3(0.015) / total:200
-------------------
training:open_rest_m_n/test:close_rest_mouse/mode:lda
model_score:0.875
mouse:112(0.56), nose:88(0.44) / total:200
-------------------
training:open_rest_m_n/test:1back_mouse/mode:lda
model_score:0.875
mouse:85(0.42), nose:115(0.57) / total:200
-------------------
training:open_rest_m_n/test:2back_mouse/mode:lda
model_score:0.875
mouse:73(0.36), nose:127(0.64) / total:200
-------------------
training:open_rest_m_n/test:open_rest_nose/mode:lda
model_score:0.875
mouse:2(0.01), nose:198(0.99) / total:200
-------------------
training:open_rest_m_n/test:close_rest_nose/mode:lda
model_score:0.9
mouse:111(0.56), nose:89(0.45) / total:200
-------------------
training:open_rest_m_n/test:1back_nose/mode:lda
model_score:0.875
mouse:91(0.46), nose:109(0.55) / total:200
-------------------
training:open_rest_m_n/test:2back_nose/mode:lda
mod